# Load Data

In [4]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

used = pd.read_csv('D:/JupyterProjects/Workspace/Practice/Data/Raw/used_car.csv',
                   index_col='Unnamed: 0')
used.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


# Preprocessing

In [5]:
# check null counts
used.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                 143
Seats                  42
New_Price            5195
Price                   0
dtype: int64

In [7]:
# drop New_Price col
used1 = used.drop(columns=['New_Price'])

In [8]:
# drop missing values from Mileage, Engine, Power columns
used2 = used1.dropna(subset=['Mileage', 'Engine', 'Power'], ignore_index=True)

In [12]:
# delete measurements from Mileage, Engine, Power columns
def drop_measure(col):
    return col.split(' ')[0]

for col in ['Mileage', 'Engine', 'Power']:
    used2[col] = used2[col].map(drop_measure)

In [13]:
# check the dtype
used2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               5874 non-null   object 
 1   Location           5874 non-null   object 
 2   Year               5874 non-null   int64  
 3   Kilometers_Driven  5874 non-null   int64  
 4   Fuel_Type          5874 non-null   object 
 5   Transmission       5874 non-null   object 
 6   Owner_Type         5874 non-null   object 
 7   Mileage            5874 non-null   object 
 8   Engine             5874 non-null   object 
 9   Power              5874 non-null   object 
 10  Seats              5872 non-null   float64
 11  Price              5874 non-null   float64
dtypes: float64(2), int64(2), object(8)
memory usage: 550.8+ KB


In [14]:
# change Mileage, Engine, Power col dtype to float, int, float
used2['Mileage'] = used2['Mileage'].astype(float)
used2['Power'] = used2['Power'].astype(float)

used2['Engine'] = used2['Engine'].astype(int)

In [23]:
# drop unnecessary columns for machine learning
used3 = used2.drop(columns=['Name', 'Location', 'Owner_Type', 'Seats'])
used3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               5874 non-null   int64  
 1   Kilometers_Driven  5874 non-null   int64  
 2   Fuel_Type          5874 non-null   object 
 3   Transmission       5874 non-null   object 
 4   Mileage            5874 non-null   float64
 5   Engine             5874 non-null   int64  
 6   Power              5874 non-null   float64
 7   Price              5874 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 367.3+ KB


In [20]:
# remove outliers
def remove_outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    
    condition = (df[col] < (q1 - 1.5 * iqr)) | (df[col] > (q3 + 1.5 * iqr))
    outlier_index = df.loc[condition, col].index
    
    if len(outlier_index) > 0:
        cleaned_col = df[col].drop(outlier_index)
        return cleaned_col
    else:
        print('No Outliers')

In [24]:
cleaned_cols =[]

for col in ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power']:
    cleaned_col = remove_outliers(used3, col)
    cleaned_cols.append(cleaned_col)

In [25]:
# create new cleaned table
used4 = pd.concat([cleaned_cols[0],
                 cleaned_cols[1], 
                 used3['Fuel_Type'], 
                 used3['Transmission'],
                 cleaned_cols[2], 
                 cleaned_cols[3],
                 cleaned_cols[4],
                 used3['Price']],
                 axis=1,
                 join='inner').reset_index(drop=True)
used4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5292 entries, 0 to 5291
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               5292 non-null   int64  
 1   Kilometers_Driven  5292 non-null   int64  
 2   Fuel_Type          5292 non-null   object 
 3   Transmission       5292 non-null   object 
 4   Mileage            5292 non-null   float64
 5   Engine             5292 non-null   int64  
 6   Power              5292 non-null   float64
 7   Price              5292 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 330.9+ KB


In [26]:
# label encoding
print(f'Fuel type: {used4['Fuel_Type'].unique()}')
print(f'Transmission: {used4['Transmission'].unique()}')

Fuel type: ['CNG' 'Diesel' 'Petrol' 'LPG']
Transmission: ['Manual' 'Automatic']


In [28]:
used5 = used4.replace({'CNG': 0,
                     'Diesel': 1,
                     'LPG': 2,
                     'Petrol': 3,
                     'Automatic': 0,
                     'Manual': 1})
used5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5292 entries, 0 to 5291
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               5292 non-null   int64  
 1   Kilometers_Driven  5292 non-null   int64  
 2   Fuel_Type          5292 non-null   int64  
 3   Transmission       5292 non-null   int64  
 4   Mileage            5292 non-null   float64
 5   Engine             5292 non-null   int64  
 6   Power              5292 non-null   float64
 7   Price              5292 non-null   float64
dtypes: float64(3), int64(5)
memory usage: 330.9 KB


In [29]:
# label encode df with outliers
used6 = used3.replace({'CNG': 0,
                     'Diesel': 1,
                     'LPG': 2,
                     'Petrol': 3,
                     'Automatic': 0,
                     'Manual': 1})
used6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               5874 non-null   int64  
 1   Kilometers_Driven  5874 non-null   int64  
 2   Fuel_Type          5874 non-null   int64  
 3   Transmission       5874 non-null   int64  
 4   Mileage            5874 non-null   float64
 5   Engine             5874 non-null   int64  
 6   Power              5874 non-null   float64
 7   Price              5874 non-null   float64
dtypes: float64(3), int64(5)
memory usage: 367.3 KB


In [31]:
# save 2 df on local folders
file_path1 = 'D:/JupyterProjects/Workspace/Practice/Data/Cleaned/used_car_cleaned.csv'
file_path2 = 'D:/JupyterProjects/Workspace/Practice/Data/Cleaned/used_car_outliers.csv'

used5.to_csv(file_path1, index=False)
used6.to_csv(file_path2, index=False)

# Load cleaned datasets

In [32]:
df1 = pd.read_csv('D:/JupyterProjects/Workspace/Practice/Data/Cleaned/used_car_cleaned.csv')
df2 = pd.read_csv('D:/JupyterProjects/Workspace/Practice/Data/Cleaned/used_car_outliers.csv')

# Split data into train data and test data

In [33]:
from sklearn.model_selection import train_test_split

X1_data = df1.iloc[:, :-1]
X2_data = df2.iloc[:, :-1]
y1_data = df1['Price']
y2_data = df2['Price']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_data,
                                                        y1_data,
                                                        test_size=.2,
                                                        random_state=0)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2_data,
                                                        y2_data,
                                                        test_size=.2,
                                                        random_state=0)